# 🏡 Airbnb Listings ETL Pipeline

This notebook performs an ETL process on Airbnb listing data.  
We will clean, transform, and prepare the data for analysis or visualization (e.g., in Tableau).

## 📥 Step 1: Load the data

We start by importing the necessary libraries and loading the raw `listings.csv` file.

In [5]:
import pandas as pd
import numpy as np
import os

# Load the raw listings dataset
listings = pd.read_csv('listings.csv')

# Display the first few rows
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,36690,Mo's Cozy Cabana,157752,Monique,Belize,Belize Islands,17.748320,-88.024740,Entire home/apt,100.0,3,461,2025-03-13,2.59,4,227,36,NaN
1,120115,"Seaside Villas 4 2nd Flr - Pool, Beach, Jacuzzi",604999,Bobbi,Belize,Belize Islands,17.741660,-88.023950,Entire home/apt,706.0,2,72,2025-02-06,0.75,18,281,10,NaN
2,120116,Valarosa - Sweet cottage,604999,Bobbi,Belize,Belize Islands,17.738116,-88.028534,Entire home/apt,172.0,2,87,2025-01-25,0.79,18,315,12,NaN
3,152512,"Toucan-kit, ac, wifi, bike, tropical gardens, ...",733857,Jacqueline,Stann Creek,Stann Creek Mainland,16.526930,-88.368640,Entire home/apt,281.0,7,73,2024-03-25,0.45,5,89,1,NaN
4,152922,rent a private room in a house,735525,Jennifer,Cayo,Cayo,17.254280,-88.767710,Private room,52.0,1,13,2025-01-14,0.12,1,365,2,NaN


## 🔎 Step 2: Inspect the dataset

We inspect the dataset structure to understand its columns, data types, and missing values.

In [6]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3433 entries, 0 to 3432
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              3433 non-null   int64  
 1   name                            3433 non-null   object 
 2   host_id                         3433 non-null   int64  
 3   host_name                       3433 non-null   object 
 4   neighbourhood_group             3433 non-null   object 
 5   neighbourhood                   3433 non-null   object 
 6   latitude                        3433 non-null   float64
 7   longitude                       3433 non-null   float64
 8   room_type                       3433 non-null   object 
 9   price                           2880 non-null   float64
 10  minimum_nights                  3433 non-null   int64  
 11  number_of_reviews               3433 non-null   int64  
 12  last_review                     27

## 🧹 Step 3: Drop unnecessary columns

We drop columns that are not useful for our analysis, such as license info or less relevant review metrics.

In [7]:
listings = listings.drop(
    labels=['license', 'availability_365', 'number_of_reviews_ltm'], 
    axis=1
)

## 🕒 Step 4: Convert date columns

We convert the `last_review` column from string to datetime format.  
Invalid parsing is handled gracefully using `errors='coerce'`.

In [8]:
listings['last_review'] = pd.to_datetime(listings['last_review'], errors='coerce')
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3433 entries, 0 to 3432
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              3433 non-null   int64         
 1   name                            3433 non-null   object        
 2   host_id                         3433 non-null   int64         
 3   host_name                       3433 non-null   object        
 4   neighbourhood_group             3433 non-null   object        
 5   neighbourhood                   3433 non-null   object        
 6   latitude                        3433 non-null   float64       
 7   longitude                       3433 non-null   float64       
 8   room_type                       3433 non-null   object        
 9   price                           2880 non-null   float64       
 10  minimum_nights                  3433 non-null   int64         
 11  numb

## 🧽 Step 5: Handle missing values

We fill missing values in the `reviews_per_month` column with `0`, assuming listings without reviews had no monthly review rate.

In [9]:
listings['reviews_per_month'] = listings['reviews_per_month'].fillna(0)
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count
0,36690,Mo's Cozy Cabana,157752,Monique,Belize,Belize Islands,17.748320,-88.024740,Entire home/apt,100.0,3,461,2025-03-13,2.59,4
1,120115,"Seaside Villas 4 2nd Flr - Pool, Beach, Jacuzzi",604999,Bobbi,Belize,Belize Islands,17.741660,-88.023950,Entire home/apt,706.0,2,72,2025-02-06,0.75,18
2,120116,Valarosa - Sweet cottage,604999,Bobbi,Belize,Belize Islands,17.738116,-88.028534,Entire home/apt,172.0,2,87,2025-01-25,0.79,18
3,152512,"Toucan-kit, ac, wifi, bike, tropical gardens, ...",733857,Jacqueline,Stann Creek,Stann Creek Mainland,16.526930,-88.368640,Entire home/apt,281.0,7,73,2024-03-25,0.45,5
4,152922,rent a private room in a house,735525,Jennifer,Cayo,Cayo,17.254280,-88.767710,Private room,52.0,1,13,2025-01-14,0.12,1


## ✍️ Step 6: Rename columns for clarity

We rename selected columns to improve clarity and consistency.

In [10]:
column_mapper = {
    'name': 'listing_name',
    'price': 'price_per_night',
    'calculated_host_listings_count': 'host_listings_count'
}

listings = listings.rename(mapper=column_mapper, axis=1)
listings.head()

,id,listing_name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price_per_night,minimum_nights,number_of_reviews,last_review,reviews_per_month,host_listings_count
0,36690,Mo's Cozy Cabana,157752,Monique,Belize,Belize Islands,17.748320,-88.024740,Entire home/apt,100.0,3,461,2025-03-13,2.59,4
1,120115,"Seaside Villas 4 2nd Flr - Pool, Beach, Jacuzzi",604999,Bobbi,Belize,Belize Islands,17.741660,-88.023950,Entire home/apt,706.0,2,72,2025-02-06,0.75,18
2,120116,Valarosa - Sweet cottage,604999,Bobbi,Belize,Belize Islands,17.738116,-88.028534,Entire home/apt,172.0,2,87,2025-01-25,0.79,18
3,152512,"Toucan-kit, ac, wifi, bike, tropical gardens, ...",733857,Jacqueline,Stann Creek,Stann Creek Mainland,16.526930,-88.368640,Entire home/apt,281.0,7,73,2024-03-25,0.45,5
4,152922,rent a private room in a house,735525,Jennifer,Cayo,Cayo,17.254280,-88.767710,Private room,52.0,1,13,2025-01-14,0.12,1


## 💾 Step 7: Save the cleaned dataset

Finally, we export the cleaned data to a new CSV file for further use (e.g., Tableau dashboard).

In [11]:
output_path = '/Users/rudolphhaink/anaconda_projects/87d98282-c864-4062-80b2-094e81b43c0b/listings_cleaned.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
listings.to_csv(output_path, index=False)

print("✅ Cleaned data saved to:", output_path)

✅ Cleaned data saved to: /Users/rudolphhaink/anaconda_projects/87d98282-c864-4062-80b2-094e81b43c0b/listings_cleaned.csv
